<a href="https://colab.research.google.com/github/huishingchong/agile_llm/blob/main/prompting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Set up
### Import Packages and API keys

In [1]:
!pip install transformers datasets torch langchain langchain-community faiss-cpu sentence-transformers python-dotenv gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 810.5/810.5 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 50.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 74.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 69.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 65.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 102.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━

In [1]:
from getpass import getpass
from dotenv import load_dotenv
import os
from pathlib import Path

env_path = Path('.') / '.env'
load_dotenv(dotenv_path=env_path)

huggingface_api_token = os.getenv('HUGGINGFACEHUB_API_TOKEN')

if not huggingface_api_token:
    huggingface_api_token = getpass("Enter your Hugging Face Hub API token: ")

In [2]:
# get a token: https://huggingface.co/docs/api-inference/quicktour#get-your-api-token
import os
from getpass import getpass

HUGGINGFACEHUB_API_TOKEN = getpass()
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN

··········


## Specify Pretrained and Fine-tuned model

In [2]:
from langchain_community.llms import HuggingFaceEndpoint
pre_trained_model_name = "tiiuae/falcon-7b"
pre_trained_llm = HuggingFaceEndpoint(
    repo_id=pre_trained_model_name,
    model=pre_trained_model_name,
    task="text-generation",
    temperature=0.5,
    max_new_tokens=200
)

fine_tuned_model_name = "tiiuae/falcon-7b-instruct"
fine_tuned_llm = HuggingFaceEndpoint(
    repo_id=fine_tuned_model_name,
    model=fine_tuned_model_name,
    task="text-generation",
    temperature=0.5,
    max_new_tokens=200
)

/Users/huishingchong/agile_llm/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /Users/huishingchong/.cache/huggingface/token
Login successful
Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /Users/huishingchong/.cache/huggingface/token
Login successful


## Template-based prompting with Langchain

In [3]:
from langchain.prompts import PromptTemplate
template= """
Please answer the question.
Answer professionally, and where appropriate, in a Computer Science educational context.
Question: {question}
Response: """

prompt = PromptTemplate(template=template, input_variables=["question"])


In [4]:
from langchain.chains import LLMChain

pre_trained_llm_chain = LLMChain(prompt=prompt, llm=pre_trained_llm, verbose=False)
fine_tuned_llm_chain = LLMChain(prompt=prompt, llm=fine_tuned_llm, verbose=False)

In [ ]:
question = "what is AI?"
input_dict = {'question': question}
response = pre_trained_llm_chain.invoke(input_dict)
response

{'question': 'what is AI?',
 'text': '(Please answer the question)\nAnswer: (Please answer the question)\nPlease use the following link to complete the survey:\nPlease let me know if you have any questions.\nThank you for your time.\nSincerely,\nDr. Richard K. Lyons\nAssociate Professor\nDepartment of Computer Science\nUniversity of North Carolina at Charlotte\n(704) 687-2606\nrichard.lyons@uncc.edu\nYou are receiving this email because you have been in contact with the Department of Computer Science at the University of North Carolina at Charlotte. If you wish to be removed from the mailing list, please reply to this email and type "remove" in the subject line.'}

In [ ]:
question = "what is AI?"
input_dict = {'question': question}
response = fine_tuned_llm_chain.invoke(input_dict)
response

{'question': 'what is AI?',
 'text': '\nAI, or Artificial Intelligence, is a branch of computer science that focuses on creating intelligent machines that can perform tasks that typically require human intelligence. These tasks include problem-solving, learning, and adapting to new situations. AI is used in many applications, such as robotics, search engines, and medical diagnosis, to name a few. In the context of a Computer Science educational program, AI is typically studied as a specialized field, with courses covering topics such as machine learning, natural language processing, and robotics.'}

From observation, the fine-tuned LLM (that was trained on chat/instruct datasets) follows and answers the prompt appropriately, however the pre-trained LLM does not seem to follow guidance from the prompt.

## Chat Interface

In [ ]:
import gradio as gr

def chat_interface(textbox, chat):
    input_dict = {'question': textbox}
    response = fine_tuned_llm_chain.invoke(input_dict)
    return response['text']

gr.ChatInterface(
    fn=chat_interface,
    chatbot=gr.Chatbot(height=300),
    textbox=gr.Textbox(placeholder="Ask me a question", container=False, scale=7),
    title="Chatbot",
    description="Ask Chatbot any question",
    theme="soft",
    examples=["What does AI stand for?", "What is Software Engineering?", "What is Cybersecurity?"],
    cache_examples=False,
    retry_btn=None,
    undo_btn="Delete Previous",
    clear_btn="Clear",
).launch(debug=True)


## Evaluation
### Comparing the performance between pre-trained and fine-tuned models with the same prompting.

In [ ]:
!pip install torch tiktoken evaluate mlflow bert-score evaluate textstat


Prepare data set

In [5]:
import pandas as pd
data_cs = pd.DataFrame({
    "question": [
        "Can you explain the concept of quantum computing and its potential applications?",
        "How does blockchain technology impact cybersecurity practices?",
        "Can you explain the difference between front-end and back-end web development?",
        "How does cloud computing influence modern software development practices?",
        "What are the key principles of agile software development methodology?",
        "Can you describe the role of DevOps in the software development lifecycle?",
        "How does natural language processing (NLP) contribute to advancements in artificial intelligence?",
        "Can you provide examples of real-world applications of computer vision technology?",
        "What are the primary principles of object-oriented programming (OOP)?",
        "How does parallel computing contribute to advancements in computer science?",

        "Concisely explain the concept of kernel machine in Machine Learning.",
        "What is the IEEE 802.11i standard?",
        "Within a paragraph, explain the MVC architecture.",
        "What are the different generations of cellular communication networks, and what is the current state of deployment for each?",
        "Provide a simple description of the Travelling Salesman Problem.",
        "What is the concept of quantum superposition in the context of quantum computing?",
        "What is the notion of NP hardness?",
        "Summarise the difference between internal and external fragmentation.",
        "Briefly explain the concept of the 'lifetime' of a variable",
        "What is a Distributed Denial of Service attack?",

        "What is a 'tag' in IoT?",
        "What type of neural network architecture is typically used for image recognition tasks, and which one is better suited for processing sequential data such as time series or text?",
        "Which sorting algorithm has the best time complexity in the worst-case scenario: Quicksort or Bubble Sort?",
        "In the context of cryptography, which encryption algorithm is considered symmetric: RSA, AES, or Diffie-Hellman?",
        "Which search algorithm guarantees to find the shortest path in a weighted graph with non-negative edge weights: Dijkstra's algorithm, A* search, or Depth-First Search (DFS)?",
        "Which database model uses a schema-less approach for data storage: Relational database, NoSQL database, or Object-Oriented database?",
        "Which data structure is typically used to implement a priority queue efficiently?",
        "True or False: The concept of 'algorithmic bias' refers to the phenomenon where algorithms systematically favor certain groups or outcomes over others.",
        "What security mechanism restricts how documents or scripts loaded from one origin can interact with resources from another origin in web browsers?",
        "What are the two primary consensus mechanisms used in blockchain networks to validate transactions and maintain the integrity of the ledger?"

    ],
    "ground_truth": [
        "Quantum computing leverages the principles of quantum mechanics to process information in ways that traditional computers cannot. Unlike classical computers that use bits to represent information as either 0 or 1, quantum computers use quantum bits or qubits, which can exist in multiple states simultaneously due to superposition and can be entangled with each other, enabling exponentially faster computations for certain problems. Potential applications of quantum computing span various domains such as cryptography, where quantum algorithms can break traditional encryption methods, and optimization, where quantum algorithms can efficiently solve complex optimization problems encountered in fields like logistics, drug discovery, and financial modeling. Furthermore, quantum machine learning holds promise for enhancing pattern recognition, data analysis, and AI algorithms. However, the field is still in its nascent stages, with practical quantum computers facing significant engineering and scalability challenges before achieving widespread adoption.",
        "Blockchain technology has a significant impact on cybersecurity practices by introducing new mechanisms for enhancing data integrity, confidentiality, and trust in digital transactions. One of the key features of blockchain is its decentralized and immutable ledger, which records transactions in a transparent and tamper-resistant manner, making it extremely difficult for malicious actors to alter or manipulate data without detection. This inherent transparency and immutability help mitigate the risk of data tampering, fraud, and unauthorized access, thereby strengthening cybersecurity defenses. Additionally, blockchain's use of cryptographic techniques ensures secure authentication, data encryption, and digital signatures, further bolstering the confidentiality and integrity of sensitive information. Furthermore, blockchain enables the development of decentralized identity management systems, where users have control over their digital identities and can securely authenticate themselves without relying on centralized authorities. Overall, blockchain technology provides innovative solutions for addressing cybersecurity challenges, offering improved data protection, integrity verification, and trustworthiness in digital transactions and interactions.",
        "Front-end web development focuses on creating the visual elements and user experience of a website or web application that users interact with directly in a web browser. Front-end developers use languages like HTML, CSS, and JavaScript to design layouts, style elements, and add interactivity. They ensure the website is visually appealing, intuitive, and responsive across devices. In contrast, back-end web development deals with the server-side logic, databases, and infrastructure that power the functionality of the website or application. Back-end developers work with languages like Python, Java, or Node.js to handle server operations, process user requests, and manage data. They implement features like user authentication, database interactions, and server-side validation to ensure security and performance. Both front-end and back-end development are essential for creating a functional and engaging web experience, with developers often specializing in one or the other or working on full-stack development, encompassing both aspects.",
        "Cloud computing has revolutionized modern software development practices by providing a scalable, flexible, and cost-effective infrastructure for building, deploying, and managing software applications. Cloud computing offers on-demand access to a wide range of computing resources, including storage, processing power, and networking, which eliminates the need for organizations to invest in and maintain their own physical hardware infrastructure. This accessibility to scalable resources enables developers to rapidly provision and deploy applications, iterate on software releases more quickly, and respond promptly to changing business requirements or user demands. Moreover, cloud platforms offer a rich ecosystem of managed services and tools for development, testing, monitoring, and deployment, streamlining the software development lifecycle and reducing time-to-market. Additionally, cloud computing facilitates collaboration among geographically distributed teams by providing centralized access to development environments and shared resources. Overall, cloud computing empowers developers with the agility, scalability, and efficiency needed to innovate and deliver high-quality software products in today's fast-paced digital landscape.",
        "Agile software development methodology is based on several key principles that emphasize flexibility, collaboration, and continuous improvement throughout the software development process. Firstly, agile values individuals and interactions over processes and tools, emphasizing the importance of effective communication and teamwork within cross-functional teams. Secondly, it prioritizes working software over comprehensive documentation, encouraging the delivery of functional software increments in short iterations, allowing for rapid feedback and adaptation to changing requirements. Thirdly, customer collaboration is valued over contract negotiation, promoting active involvement and feedback from stakeholders throughout the development cycle to ensure the delivered product meets their needs. Additionally, agile emphasizes responding to change over following a rigid plan, embracing change as a natural part of the development process and adapting quickly to new requirements or insights. Overall, these principles guide agile teams in delivering high-quality software that meets customer expectations through iterative development, collaboration, and flexibility.",
        "DevOps plays a crucial role in the software development lifecycle by bridging the gap between development (Dev) and operations (Ops) teams, with the goal of accelerating software delivery while maintaining high quality and reliability. DevOps promotes collaboration, communication, and automation throughout the development process, from code creation to deployment and maintenance. It emphasizes practices such as continuous integration (CI) and continuous delivery (CD), where code changes are frequently integrated, tested, and deployed in automated pipelines, enabling rapid feedback and iteration. DevOps also encourages infrastructure as code (IaC), where infrastructure configurations are managed programmatically, allowing for consistent, repeatable deployments and scalability. By fostering a culture of collaboration, automation, and continuous improvement, DevOps enables teams to deliver software faster, more reliably, and with reduced risk, ultimately enhancing the efficiency and effectiveness of the software development lifecycle.",
        "Natural language processing (NLP) plays a pivotal role in advancing artificial intelligence (AI) by enabling machines to understand, interpret, and generate human language. Through the application of algorithms and techniques from computational linguistics and machine learning, NLP allows computers to analyze and derive meaning from unstructured text data, such as written or spoken language. This capability opens up a wide array of applications across various domains, including language translation, sentiment analysis, chatbots, virtual assistants, and information extraction. By harnessing the power of NLP, AI systems can interact with humans in a more natural and intuitive manner, facilitating communication and collaboration between humans and machines. Moreover, NLP enhances AI's ability to process and derive insights from vast amounts of textual data, leading to advancements in fields like data analytics, knowledge discovery, and decision support. Overall, NLP serves as a cornerstone of AI development, driving innovation and unlocking new possibilities for human-computer interaction and intelligent automation.",
        "Computer vision technology has numerous real-world applications across various industries. In healthcare, it's used for medical image analysis, enabling automated diagnosis and detection of diseases from X-rays, MRIs, and CT scans, improving patient outcomes and treatment planning. In retail, computer vision powers cashier-less checkout systems, smart shelves, and facial recognition for personalized shopping experiences and inventory management. In autonomous vehicles, computer vision algorithms analyze road scenes, detect obstacles, and interpret traffic signs to facilitate safe navigation and driving decisions. In manufacturing, it's utilized for quality inspection, defect detection, and robotic automation, enhancing production efficiency and product quality. Additionally, computer vision is employed in security and surveillance systems for facial recognition, object tracking, and anomaly detection, bolstering public safety and crime prevention efforts. These examples illustrate the diverse applications and transformative impact of computer vision technology across various domains.",
        "Object-oriented programming (OOP) is based on four primary principles: encapsulation, inheritance, polymorphism, and abstraction. These principles enable developers to create modular, reusable, and maintainable software systems by organizing code around objects and their interactions.",
        "Parallel computing involves executing multiple computational tasks simultaneously, leading to increased performance and efficiency in solving complex problems. It is essential in various fields of computer science, such as scientific computing, data analysis, and artificial intelligence, where large-scale computations are required.",

        "In machine learning, a kernel machine is a type of algorithm that operates by implicitly mapping input data into a higher-dimensional space where it becomes easier to separate different classes or categories. This mapping is done using a mathematical function called a kernel, which calculates the similarity or distance between data points in the original input space. Kernel machines are commonly used in support vector machines (SVMs) and kernel ridge regression, among other algorithms. They allow for complex decision boundaries to be learned from data that may not be linearly separable in the original input space. By effectively transforming data into a higher-dimensional space without explicitly computing the transformation, kernel machines enable powerful learning algorithms to be applied to a wide range of machine learning tasks.",
        "IEEE 802.11i is a standard for wireless network security, specifically focusing on enhancing the security protocols used in Wi-Fi networks. It is also commonly known as WPA2 (Wi-Fi Protected Access 2). IEEE 802.11i was ratified in 2004 and is an enhancement of the original WEP (Wired Equivalent Privacy) protocol.",
        "The Model-View-Controller (MVC) architecture is a software design pattern commonly used in developing web applications. In MVC, the application is divided into three interconnected components: the Model, the View, and the Controller. The Model represents the data and business logic of the application, encapsulating its state and behavior. It communicates with the database, processes data, and responds to requests from the Controller. The View is responsible for presenting the data to the user, providing the interface through which users interact with the application. It receives data from the Model and renders it in a user-friendly format, such as HTML for web applications. The Controller acts as an intermediary between the Model and the View, handling user input and requests, invoking appropriate actions on the Model, and determining which View to display in response. This separation of concerns in MVC promotes modularity, maintainability, and scalability in software development, allowing for easier management and modification of individual components without affecting the overall functionality of the application.",
        "Cellular communication networks are categorized into generations, represented by numeric designations: 1G, 2G, 3G, 4G, and the emerging 5G. Currently, widespread deployment is established in 4G networks, with 5G steadily gaining traction. Each successive generation introduces advancements in mobile telecommunications technology, offering improved data speeds, network capacity, and features.",
        "The Travelling Salesman Problem (TSP) is a classic problem in computer science and optimization. In simple terms, it asks: 'Given a list of cities and the distances between them, what is the shortest possible route that visits each city exactly once and returns to the original city?' The challenge is to find the most efficient route that minimizes the total distance traveled. The problem is known to be NP-hard, meaning that as the number of cities increases, finding the optimal solution becomes increasingly difficult and computationally expensive.",
        "In quantum computing, quantum superposition refers to the ability of quantum bits, or qubits, to exist in multiple states simultaneously. Unlike classical bits, which can only be in a state of 0 or 1 at any given time, qubits can be in a state that is a combination of 0 and 1 simultaneously, thanks to the principles of quantum mechanics. This superposition property allows quantum computers to perform multiple calculations in parallel, potentially leading to significant speedups for certain types of problems when compared to classical computers.",
        "The notion of NP-hardness refers to a computational problem that is at least as hard as the hardest problems in NP (Nondeterministic Polynomial time), which includes problems that can be solved in polynomial time by a nondeterministic Turing machine. NP-hard problems are not necessarily in NP themselves, but any problem in NP can be reduced to an NP-hard problem in polynomial time. Therefore, solving an NP-hard problem would imply solving all problems in NP efficiently. These problems are considered computationally challenging and often require exponential time to solve.",
        "Internal fragmentation occurs when allocated memory blocks within a process are larger than necessary, resulting in wasted memory space due to unused portions within the allocated blocks. This typically happens in fixed-size memory allocation schemes where processes are assigned memory blocks that may be larger than the actual size required by the process, leading to inefficient use of memory resources. On the other hand, external fragmentation occurs when there is unallocated memory space scattered throughout the memory, making it difficult to allocate contiguous blocks of memory to processes even though the total free memory space may be sufficient. This fragmentation occurs over time as memory is allocated and deallocated, leaving gaps or holes of various sizes between allocated blocks. External fragmentation can lead to inefficient memory utilization and can be particularly problematic in dynamic memory allocation schemes.",
        "The lifetime of a variable is the period of time in which the variable can be used while the program is running. It is the period of time in which the variable has been allocated a memory address and has not yet been deallocated.",
        "A Distributed Denial of Service (DDoS) attack is a malicious attempt to disrupt the normal functioning of a targeted server, service, or network by overwhelming it with a flood of traffic from multiple sources. In a DDoS attack, a large number of compromised devices, often referred to as 'bots' or 'zombies', are orchestrated by an attacker to send an excessive volume of requests or data packets to the target, rendering it inaccessible or significantly slowing down its performance. DDoS attacks can take various forms, including volumetric attacks that flood the target with massive amounts of traffic, protocol attacks that exploit vulnerabilities in network protocols, and application layer attacks that target specific applications or services. The goal of a DDoS attack is to disrupt the availability of the targeted resource, causing inconvenience, financial loss, or reputational damage to the victim organization.",

        "In IoT, a 'tag' typically refers to a small, low-power device equipped with sensors and a unique identifier (such as RFID or NFC) used for tracking and monitoring assets, goods, or objects. Tags are often passive or semi-passive devices that rely on external readers or sensors to transmit data.",
        "CNNs are commonly employed for tasks involving image recognition, whereas RNNs are well-suited for processing sequential data such as time series or text.",
        "Quicksort has a time complexity of O(n log n) while Bubble Sort has a time complexity of O(n^2). Therefore, Quicksort has better time complexity in the worst-case.",
        "In the context of cryptography, the encryption algorithm considered symmetric is AES (Advanced Encryption Standard).",
        "Dijkstra's algorithm guarantees to find the shortest path in a weighted graph with non-negative edge weights.",
        "The NoSQL database model uses a schema-less approach for data storage.",
        "A binary heap is typically used to implement a priority queue efficiently.",
        "True. The concept of 'algorithmic bias' refers to the phenomenon where algorithms systematically favor certain groups or outcomes over others, often due to the presence of biased training data or inherent biases in the algorithm design. It is important to review training data and act upon any algorithm bias identified to ensure fairness and equity in our machine learning or AI models.",
        "The same-origin policy is a crucial security measure in web browsers that prevents scripts from one origin from accessing resources from another origin. By enforcing origin integrity, it ensures that scripts loaded from one source cannot interact with resources loaded from a different source. This is essential for safeguarding sensitive data and resources against potential malicious access. Implemented across major browsers like Chrome, Firefox, and Safari, the same-origin policy serves as a foundational security feature, upholding the principle of isolating origins to mitigate security risks.",
        "Proof of Work (PoW) and Proof of Stake (PoS) are the two main consensus mechanisms used in blockchain networks. In PoW, nodes compete to solve complex mathematical problems to validate blocks of transactions, with the first miner to solve it being rewarded with cryptocurrency. This process enhances network security and ensures the immutability of the ledger. On the other hand, PoS requires nodes to stake cryptocurrency to validate transactions, incentivizing them to maintain the network. Nodes are mandated to hold a certain amount of cryptocurrency, promoting network participation and transaction validation."
    ]
})

data_cs_industry = pd.DataFrame({
    "question": [
        "What are the latest trends in artificial intelligence and machine learning?",
        "What are some emerging programming languages that are gaining popularity in the industry?",
        "I am a beginner that wants to get into Data Science, where should I start?",
        "I am a final-year Computer Science student wanting to find a graduate role in Cybersecurity. What are the practical skills required for a career in Cybersecurity that are currently in-demand?",
        "What are the essential skills required for a career in cybersecurity?",
        "What are some in-demand technical skills for aspiring data analysts?",
        "What are the career prospects for individuals with expertise in cybersecurity risk management?",
        "What are new job roles created in the AI industry in recent years?",
        "What are the main responsibilities of a web developer in computer science careers?",
        "What is the job of a software quality assurance (QA) engineer?",

        "What are the three main languages for web development?",
        "Tell me about the newest programming languages of 2024.",
        "What are some popular software architecture frameworks that a professional in software engineering should know and give a brief description of each.",
        "What is an eSIM and how does it differ to the traditional physical SIM card?",
        "What relevant qualifications does an IT manager need?",
        "Why are certifications required for database administrator and give an example of a recent certification.",
        "Give examples of tools that are essential for an IT consultant.",
        "What are the daily challenges faced often by database administrators?",
        "What are some educational requirements for a career in Data Analyst?",
        "Give a list of top certifications for a Penetration Tester.",

        "How does job description for a role in software engineering differ for full-stack software engineer?",
        "What qualifications does a senior software engineer need that recruiters look for compared to a regular software engineer level?",
        "What automatic differentiation libraries does a professional in AI typically need to know?",
        "What are the difference in resources used between front-end and back-end web developing?",
        "How can one develop a career into digital forensics?",
        "What are the specific skill set for a network engineer?",
        "Please give me advice to become a full-stack developer.",
        "Explain what a CTO does and some common responsibilities they have.",
        "Can you discuss the role of a machine learning engineer, and what algorithms, libraries, and frameworks such as TensorFlow and PyTorch are commonly used for developing machine learning models?",
        "What qualifications and certifications are important for a cloud computing architect, and what cloud platforms such as AWS, Azure, and Google Cloud are commonly used in this role?"
    ],
    "ground_truth": [
        "In the realm of artificial intelligence (AI) and machine learning (ML), several notable trends have emerged recently. Firstly, there's a growing focus on explainable AI (XAI), which aims to make AI models more transparent and understandable to humans, crucial for applications in fields like healthcare and finance where interpretability is paramount. Secondly, federated learning has gained traction, enabling training of ML models across decentralized devices while preserving data privacy, pivotal for IoT and edge computing scenarios. Additionally, reinforcement learning (RL) advancements, particularly in deep RL, have seen remarkable progress, empowering AI systems to make sequential decisions in dynamic environments, with applications spanning robotics, autonomous vehicles, and gaming. Lastly, the integration of AI with other technologies like blockchain for enhanced security and trustworthiness and with quantum computing for tackling complex optimization problems signifies promising directions for future research and innovation in the AI landscape.",
        "Several emerging programming languages are gaining traction in the industry due to their unique features and capabilities. One such language is Rust, known for its emphasis on safety, concurrency, and performance, making it suitable for systems programming where reliability and efficiency are critical. Another language on the rise is Julia, which specializes in numerical and scientific computing, offering high performance comparable to traditional languages like C and Fortran while maintaining a user-friendly syntax and extensive library support. Additionally, Kotlin, a statically typed language interoperable with Java, has become increasingly popular for Android app development, offering modern features and improved developer productivity. Lastly, Swift, developed by Apple, has gained momentum for iOS and macOS development, providing a concise and expressive syntax along with powerful features like optionals and automatic memory management. These emerging languages cater to specific niches and address evolving industry needs, showcasing their growing relevance and adoption in the programming landscape.",
        "Here is a few things to learn about Data Science to get you started: \nLearn Python or R: Choose one as your primary programming language. \nBasic Statistics: Understand mean, median, mode, standard deviation, and probability.\nData Manipulation: Learn Pandas (Python) or dplyr (R) for data cleaning and manipulation.\nData Visualization: Use Matplotlib, Seaborn (Python), or ggplot2 (R) for visualization.\nMachine Learning Basics: Start with linear regression, logistic regression, decision trees, and evaluation metrics.\nPractice: Work on projects using real-world datasets from sources like Kaggle.\nStay Updated: Follow online resources and communities for the latest trends and techniques.",
        "As a final-year Computer Science student aiming for a graduate role in cybersecurity, it's essential to focus on developing practical skills that are currently in high demand in the industry.\nSome of these key skills include:\n\n1. Knowledge of Networking: Understanding networking fundamentals, protocols (such as TCP/IP), and network architecture is crucial for identifying and mitigating security threats. Familiarize yourself with concepts like firewalls, routers, VPNs, and intrusion detection systems (IDS).\n\n2. Proficiency in Operating Systems: Gain proficiency in operating systems such as Linux and Windows, including command-line operations, system administration tasks, and security configurations. Being able to secure and harden operating systems is essential for protecting against common cybersecurity threats.\n\n3. Understanding of Cryptography: Cryptography is at the heart of cybersecurity, so having a solid understanding of encryption algorithms, cryptographic protocols, and cryptographic techniques is vital. Learn about symmetric and asymmetric encryption, digital signatures, hashing algorithms, and their applications in securing data and communications.\n\n4. Penetration Testing and Ethical Hacking: Develop skills in penetration testing and ethical hacking to identify vulnerabilities and assess the security posture of systems and networks. Familiarize yourself with tools and techniques used by ethical hackers, such as Kali Linux, Metasploit, Nmap, and Wireshark.\n\n5. Security Assessment and Risk Management: Learn how to conduct security assessments, risk assessments, and threat modeling to identify, prioritize, and mitigate security risks effectively. Understand risk management frameworks like NIST, ISO 27001, and COBIT, and how to apply them in real-world scenarios.\n\n6. Incident Response and Forensics: Acquire knowledge of incident response procedures, including detection, analysis, containment, eradication, and recovery from security incidents. Understand digital forensics principles and techniques for investigating and analyzing security breaches and cybercrimes.\n\n7. Security Awareness and Communication: Develop strong communication skills to effectively convey cybersecurity concepts, risks, and recommendations to technical and non-technical stakeholders. Being able to raise awareness about cybersecurity best practices and policies is essential for promoting a security-conscious culture within organizations.\n\n8. Continuous Learning and Adaptability: Cybersecurity is a rapidly evolving field, so it's essential to cultivate a mindset of continuous learning and adaptability. Stay updated with the latest threats, trends, technologies, and best practices through professional development, certifications, and participation in cybersecurity communities and events.\n\nBy focusing on developing these practical skills and staying abreast of industry trends and advancements, you'll be well-prepared to pursue a successful career in cybersecurity upon graduation. Additionally, consider obtaining relevant certifications such as CompTIA Security+, CEH (Certified Ethical Hacker), CISSP (Certified Information Systems Security Professional), or others to further enhance your credentials and marketability in the field.",
        "A career in cybersecurity require a broad spectrum of practical skills, including proficiency in network security protocols and tools like firewalls and intrusion detection/prevention systems (IDS/IPS) for safeguarding network infrastructure. Secure coding practices and knowledge of common vulnerabilities are essential for developing secure software applications, with expertise in frameworks like OWASP Top 10 aiding in vulnerability mitigation. Encryption techniques and cryptographic protocols are vital for securing sensitive data, while incident response and digital forensics skills, alongside tools like SIEM systems, enable effective threat detection and response. Proficiency in penetration testing frameworks like Metasploit and security assessment tools is crucial for identifying and remediating security weaknesses, while knowledge of compliance frameworks such as GDPR ensures organizational adherence to cybersecurity regulations. Effective communication and collaboration skills are imperative for conveying cybersecurity risks and recommendations to stakeholders and collaborating with cross-functional teams to implement security measures. Continued learning and staying updated with the latest cybersecurity trends and technologies are key for navigating this ever-evolving field successfully.",
        "In-demand technical skills for data analysts include proficiency in programming languages like Python, R, or SQL for data manipulation, analysis, and visualization. Familiarity with statistical analysis techniques, such as regression analysis, hypothesis testing, and predictive modeling, is essential for deriving insights from data. Knowledge of data querying and database management systems like MySQL, PostgreSQL, or MongoDB is valuable for accessing and organizing large datasets. Expertise in data wrangling techniques, using tools like pandas, dplyr, or data.table, enables cleaning and transforming raw data into actionable insights. Proficiency in data visualization libraries like Matplotlib, ggplot2, or seaborn is crucial for creating informative and visually appealing charts, graphs, and dashboards to communicate findings effectively. Additionally, experience with machine learning frameworks like scikit-learn or TensorFlow, along with knowledge of data mining techniques, enhances the ability to build predictive models and extract patterns from data.",
        "Career opportunities for individuals in cybersecurity risk management include roles such as cybersecurity risk analysts, security consultants, risk managers, compliance officers, and cybersecurity architects. These professionals play a critical role in identifying, evaluating, and prioritizing cybersecurity risks, developing risk mitigation strategies, and ensuring compliance with regulatory requirements and industry standards. With the ever-evolving threat landscape and the increasing complexity of cybersecurity challenges, individuals with expertise in cybersecurity risk management can expect to have a wide range of career opportunities and advancement prospects in both the public and private sectors, including government agencies, financial institutions, healthcare organizations, and consulting firms. Additionally, obtaining relevant certifications such as Certified Information Systems Security Professional (CISSP), Certified Information Security Manager (CISM), or Certified Risk and Information Systems Control (CRISC) can further enhance career prospects and credibility in the field.",
        "In recent years, the rapid advancement of artificial intelligence (AI) technologies has led to the emergence of several new roles within the AI industry. One such role is that of the AI Ethics Officer, responsible for ensuring that AI systems are developed and deployed in an ethical and responsible manner, addressing concerns related to bias, fairness, transparency, and accountability. Additionally, AI Product Managers play a crucial role in overseeing the development and implementation of AI-driven products and services, from ideation to launch, ensuring alignment with business objectives and user needs. Another emerging role is that of the AI Solutions Architect, tasked with designing and implementing scalable and efficient AI solutions tailored to specific business challenges and requirements. Furthermore, AI Security Specialists focus on safeguarding AI systems and data from cyber threats and vulnerabilities, addressing the unique security challenges posed by AI technologies. These new roles reflect the growing importance of ethics, governance, and interdisciplinary collaboration in the AI industry, as organizations seek to harness the transformative potential of AI while mitigating risks and ensuring responsible AI deployment.",
        "Web developers design, build, and maintain websites and web applications, including front-end and back-end development, user interface design, and database integration, to deliver a seamless user experience.",
        "Software QA engineers develop and execute test plans, scripts, and procedures to ensure the quality, reliability, and performance of software applications and systems before release to production. They identify and report defects and work closely with development teams to resolve issues and improve software quality.",

        "A web developer should know these three main programming languages: HTML, CSS, and JavaScript.",
        "In the realm of programming languages, several newcomers have been causing a stir in the tech industry. Mojo, for instance, emerged onto the scene with claims of speeds up to 35,000 times faster than Python, capturing attention with its performance capabilities. Rust, born out of frustration with glitch-laden code, has swiftly gained traction for its focus on memory safety and reliability, attracting millions of users worldwide. Julia, on the other hand, has carved a niche for itself in numerical computing, machine learning, and network security, thanks to its versatility, clear syntax, and practicality. Meanwhile, Swift, developed by Apple, has been steadily rising in popularity, lauded for its intuitive design, low learning curve, and features aimed at reducing coding errors and crashes. These languages represent the forefront of innovation and efficiency in the programming landscape of 2024.",
        "Layered Architecture: Organizes the software system into layers with specific responsibilities and dependencies, enhancing maintainability and isolating issues for easier debugging. However, it may introduce performance and complexity issues due to rigid hierarchy and coupling of layers.\n\nMicroservices Architecture: Breaks down the software system into small, independent services communicating through APIs, enabling high scalability and faster development cycles. Requires more coordination and testing but offers resilience and availability.\n\nEvent-Driven Architecture: Relies on events to trigger software component actions, supporting high responsiveness and flexibility. However, it can increase system complexity and difficulty in debugging.\n\nHexagonal Architecture: Aims to isolate core logic from external dependencies using core, ports, and adapters. Enhances testability and maintainability but adds layers and abstraction to the system.\n\nServerless Architecture: Leverages cloud services to run software components without servers, reducing operational costs and development time. However, it introduces dependency and latency issues and challenges in testing and monitoring.",
        "An eSIM, or embedded Subscriber Identity Module, is a digital SIM card embedded directly into a device, such as a smartphone, tablet, or wearable device. Unlike traditional physical SIM cards, which need to be inserted and removed from the device, an eSIM is built into the device's hardware and can be activated, provisioned, and managed remotely by the user or the mobile network operator. eSIM technology enables users to switch between mobile network carriers without needing to physically swap SIM cards, making it more convenient for users to connect to different networks while traveling or switching between service providers.",
        "For an IT manager role, a relevant qualification typically includes a degree in an IT-related field or a business degree with technical elements like mathematics or engineering. Experience demonstrating increasing responsibility in project development and implementation is valuable, even without a degree. Those transitioning from another sector should highlight project management skills, preferably in IT-related contexts, and bolster technical knowledge through IT courses. PRINCE2 (Projects in Controlled Environments) certification is increasingly sought after. Ongoing training, especially in project management, is crucial in the IT industry, with a focus on both technical and business skills. For IT managers with IT backgrounds, project management qualifications from associations like APM, ISEB, or PMI are beneficial. Alternatively, those from a general business background may pursue generic project management qualifications or courses focusing on communication and leadership skills.",
        "Certifications are required for database administrators to demonstrate expertise in specific database software platforms. These certifications validate their understanding and proficiency in managing and maintaining databases. Examples of such certifications include:\n\n- IBM Certified Database Administrator: Covers DB2 platform and SQL.\n- Microsoft Certified Solutions Associate: SQL Database Administration: Focuses on SQL and database administration.\n- Oracle Database Administrator Certified Associate: Requires completion of courses and passing an exam.\n\nA recent example is CompTIA DataSys+, which validates skills in deploying, managing, and maintaining databases, including scripting, security best practices, and business continuity strategies.",
        "These are some of the tools that are essential for an IT consultant. Of course, depending on your specific area of expertise and the nature of your projects, you may need to use other tools as well. The key is to find the tools that suit your needs and preferences, and that can help you deliver high-quality results for your clients.\n\n1 Data Analysis Tools: Microsoft Excel, Tableau, Power BI, R, Python, and SQL.\n2 Project Management Tools: Microsoft Project, Trello, Asana, Jira, and Basecamp.\n3 Communication Tools: Microsoft Teams, Zoom, Slack, Skype, and Google Workspace.\n4 Security Tools: Bitlocker, Acronis, Norton, Malwarebytes, and Windows Defender.\n5 Troubleshooting Tools: Windows Task Manager, Device Manager, Event Viewer, Ping, and Traceroute.",
        "Database administrators often encounter daily challenges related to cloud migration, managing large database systems, ensuring secure data flow, migrating to new technology, frequent deployments, and consistent performance improvement. They must navigate hybrid database systems combining cloud and on-premises implementations, manage the complexities of maintaining large databases, and prioritize secure data flow to prevent breaches. Staying abreast of rapid technological advancements is essential, as is managing frequent deployments while minimizing errors. Additionally, database administrators continually strive to optimize performance to meet evolving business needs.",
        "For a career in Data Analyst, educational requirements typically include a bachelor's degree in a related field like data analysis or computer science, though some employers may prefer candidates with a master's degree for advanced roles and better prospects. Proficiency in data analysis tools like Excel, SQL, and Python is essential, along with experience working with databases. Completing internships during education can also enhance credentials and practical skills.",
        "CompTIA:\n\nCompTIA PenTest+\nCompTIA Cybersecurity Analyst (CySA+)\nCompTIA Advanced Security Practitioner (CASP+) Optional Voucher\n\nEC-Council:\n\nCertified Ethical Hacker (CEH)\nCertified Penetration Tester (CPT)\nCertified Expert Penetration Tester (CEPT)\nEC-Council Licensed Penetration Tester (LPT) Master\n\nGIAC:\n\nGlobal Information Assurance Certification (GIAC) Penetration Tester (GPEN)",

        "A job description for a full-stack software engineer typically includes responsibilities for both frontend and backend development, while other software engineering roles may focus on specialized areas such as backend, frontend, DevOps, or data engineering.",
        "Qualifications that recruiters look for in a senior software engineer compared to a regular software engineer include both bachelor’s and master’s degrees in fields like computer science. This position commonly requires around six years of experience, but some organizations can require upwards of 10 years for their senior software engineers. Optional but beneficial certifications in relevant technologies or methodologies, such as AWS Certified Solutions Architect, Scrum Master, or Google Professional Cloud Developer.",
        "A professional in AI typically needs to know various automatic differentiation libraries to efficiently build and train machine learning models. Some commonly used libraries include TensorFlow, PyTorch, JAX, MXNet, and Autograd. These libraries provide automatic differentiation capabilities, allowing for easy experimentation and efficient training of machine learning models.",
        "Front-end web development primarily utilizes HTML, CSS, and JavaScript to structure, style, and add interactivity to web pages. HTML provides the basic structure, CSS enhances visual styling, and JavaScript adds dynamic elements.\n\nFront-end developers also utilize libraries and frameworks such as AngularJS, jQuery, and React to streamline development processes and enhance functionality.\n\nOn the other hand, back-end web development involves programming languages like Python, Java, and C#, along with database management using SQL and tools like MySQL.\n\nBack-end developers focus on server-side logic and data management, often utilizing libraries and frameworks like Django, Ruby on Rails, Flask, and Laravel to expedite development and ensure scalability.\n\nOverall, front-end development is more focused on user interface and experience, while back-end development deals with server-side processing and data management.",
        "Breaking into the field of digital forensics requires a combination of education, experience, and technical skills. A degree in computer science, cybersecurity, IT, or forensic science can provide a solid foundation. Obtaining certifications such as Certified Forensic Computer Examiner (CFCE) or GIAC Certified Forensic Analyst (GCFA) can validate skills and knowledge. Gaining experience through internships, mentorships, and volunteer work is helpful, as well as building technical skills in network security, coding languages, and data analysis. Staying current on industry trends and new technology is crucial for remaining competitive.",
        "Network engineers require a diverse skill set to excel in their roles. They need a solid understanding of network infrastructure and hardware, constantly adapting to advancements in technology. Common skills include proficiency in configuration management tools like Ansible and Puppet, understanding APIs for software integration, knowledge of cloud architecture for efficient resource utilization, and expertise in NetDevOps to integrate network automation into application development processes. Network automation, programming, and software-defined networking are essential for efficient network management and operation. Additionally, familiarity with virtualization technologies enables efficient resource utilization, a cornerstone of modern network engineering practices.",
        "Here are some advice to become a full stack developer:\n1. Define the type of developer you aspire to be.\n2. Acquire a comprehensive background in computer science covering front end, back end, database, dev-ops, and mobile app development.\n3. Embrace the challenge of coding extensively.\n4. Build a diverse coding portfolio with projects spanning various programming languages.\n5. Develop proficiency in front end and back end languages such as HTML, CSS, JavaScript, PHP, Python, and Ruby.\n6. Utilize platforms like GitHub for code storage and collaboration.\n7. Cultivate soft skills essential for developers, including problem-solving, attention to detail, creativity, and effective communication.\nThese steps form a roadmap towards becoming a competent full stack developer, encompassing both technical and interpersonal skills crucial for success in the field.",
        "The Chief Technology Officer (CTO) holds a pivotal role in companies, overseeing all technological decisions and directly reporting to top executives like the CEO or CIO. Their main task is to leverage technology to achieve business objectives efficiently. Common responsibilities include leading technology teams, devising technology strategies, managing hardware and software assets, determining system updates, evaluating new products, and guiding strategic decisions. Additionally, CTOs stay updated on digital trends, ensure compliance with privacy and security regulations, and provide regular reports to company leadership.",
        "The role of a machine learning engineer involves designing, implementing, and deploying machine learning models to solve real-world problems. Machine learning engineers work closely with data scientists and software engineers to collect and preprocess data, select appropriate algorithms and models, train and evaluate machine learning models, and deploy them into production environments. Commonly used algorithms in machine learning include linear regression, logistic regression, decision trees, random forests, support vector machines, k-nearest neighbors, neural networks, and more. Libraries and frameworks such as TensorFlow and PyTorch are commonly used for developing machine learning models. TensorFlow, developed by Google, is an open-source machine learning framework widely used for building and training deep learning models. PyTorch, developed by Facebook, is another popular open-source machine learning library known for its flexibility and ease of use, particularly for research and prototyping of neural networks.",
        "Qualifications and certifications important for a cloud computing architect typically include:\n\n1. Bachelor's degree in computer science, information technology, or a related field.\n2. Professional certifications from cloud service providers such as AWS Certified Solutions Architect, Microsoft Certified: Azure Solutions Architect Expert, or Google Cloud Certified: Professional Cloud Architect.\n3. Hands-on experience with cloud computing platforms and services, including infrastructure as a service (IaaS), platform as a service (PaaS), and software as a service (SaaS).\n4. Proficiency in networking, security, and system administration concepts relevant to cloud environments.\n5. Strong problem-solving and analytical skills, with the ability to design and implement scalable, reliable, and cost-effective cloud solutions.\n\nCommonly used cloud platforms in this role include:\n\n1. Amazon Web Services (AWS): A leading cloud services provider offering a wide range of services including compute, storage, database, machine learning, and more.\n2. Microsoft Azure: A cloud computing platform by Microsoft that provides services such as virtual machines, databases, AI, and IoT.\n3. Google Cloud Platform (GCP): Google's cloud services platform offering similar services to AWS and Azure, including compute, storage, machine learning, and data analytics."
    ]
})

Run the models over the evaluation dataset once and save results to their respective csv file (for static evaluation)

In [6]:
# Helper function: To prevent having to run predictions again, save output to the dataset as a new column 'predictions'
import datasets

def chain_predictions_from_data(llm_chain, eval_data):
  data = eval_data.copy()
  predictions = []  # List to store predictions for all questions
  for question in data['question']:
    # Invoke the llm_chain model with the current question
    input_dict = {'question': question}
    response = llm_chain.invoke(input_dict)
    generated_text = response['text']
    predictions.append(generated_text)  # Append the generated text to the predictions list

  data['predictions'] = predictions  # Assign the predictions list to a new column
  return data

In [8]:
pre_data_cs = chain_predictions_from_data(pre_trained_llm_chain, data_cs)
pre_data_cs_industry = chain_predictions_from_data(pre_trained_llm_chain, data_cs_industry)

# Create folders if they don't exist
os.makedirs("cs", exist_ok=True)
os.makedirs("industry", exist_ok=True)

pre_data_cs.to_csv("cs/pre-trained.csv", index=False)
pre_data_cs_industry.to_csv("industry/pre-trained.csv", index=False)

In [9]:
fine_data_cs = chain_predictions_from_data(fine_tuned_llm_chain, data_cs)
fine_data_cs_industry = chain_predictions_from_data(fine_tuned_llm_chain, data_cs_industry)

fine_data_cs.to_csv("cs/fine-tuned.csv", index=False)
fine_data_cs_industry.to_csv("industry/fine-tuned.csv", index=False)

Generate bert score results table

In [10]:
from evaluate import load

def generate_bert_results_table(eval_data):
    ref_texts = eval_data['ground_truth']
    predictions = eval_data['predictions']

    # Compute BERTScore
    bertscore = load("bertscore")
    results = bertscore.compute(predictions=predictions, references=ref_texts, model_type="distilbert-base-uncased")

    # Create DataFrame from BERTScore results
    bert_results_table = pd.DataFrame(results, index=range(0, len(results['precision'])))
    bert_results_table['question'] = eval_data['question']

    bert_results_table.drop(columns=['hashcode'], inplace=True)
    bert_results_table = bert_results_table[['question', 'precision', 'recall', 'f1']]

    # Calculate average precision, recall, and F1 scores
    avg_precision = bert_results_table['precision'].mean()
    avg_recall = bert_results_table['recall'].mean()
    avg_f1 = bert_results_table['f1'].mean()

    # Add a row at the bottom for average scores
    avg_row = pd.DataFrame({'question': 'Average', 'precision': avg_precision, 'recall': avg_recall, 'f1': avg_f1}, index=[len(bert_results_table)])
    bert_results_table = pd.concat([bert_results_table, avg_row])

    return bert_results_table

In [11]:
generate_bert_results_table(pre_data_cs)

,question,precision,recall,f1
0,Can you explain the concept of quantum computi...,0.797644,0.821821,0.809552
1,How does blockchain technology impact cybersec...,0.773978,0.781408,0.777675
2,Can you explain the difference between front-e...,0.841776,0.793036,0.816679
3,How does cloud computing influence modern soft...,0.806154,0.805379,0.805766
4,What are the key principles of agile software ...,0.738345,0.745066,0.741690
5,Can you describe the role of DevOps in the sof...,0.808232,0.742065,0.773736
6,How does natural language processing (NLP) con...,0.692713,0.698629,0.695659
7,Can you provide examples of real-world applica...,0.577685,0.545853,0.561318
8,What are the primary principles of object-orie...,0.700249,0.778929,0.737496
9,How does parallel computing contribute to adva...,0.613949,0.755305,0.677331


In [12]:
generate_bert_results_table(fine_data_cs)

,question,precision,recall,f1
0,Can you explain the concept of quantum computi...,0.863896,0.857400,0.860636
1,How does blockchain technology impact cybersec...,0.838449,0.824378,0.831354
2,Can you explain the difference between front-e...,0.873297,0.831687,0.851984
3,How does cloud computing influence modern soft...,0.862975,0.837650,0.850124
4,What are the key principles of agile software ...,0.775448,0.735810,0.755109
5,Can you describe the role of DevOps in the sof...,0.837935,0.810928,0.824210
6,How does natural language processing (NLP) con...,0.875801,0.838155,0.856565
7,Can you provide examples of real-world applica...,0.797935,0.815586,0.806664
8,What are the primary principles of object-orie...,0.816757,0.890041,0.851826
9,How does parallel computing contribute to adva...,0.806230,0.864826,0.834500


In [13]:
generate_bert_results_table(pre_data_cs_industry)

,question,precision,recall,f1
0,What are the latest trends in artificial intel...,0.716560,0.733428,0.724896
1,What are some emerging programming languages t...,0.758054,0.735855,0.746789
2,I am a beginner that wants to get into Data Sc...,0.699246,0.623376,0.659135
3,I am a final-year Computer Science student wan...,0.779652,0.745023,0.761944
4,What are the essential skills required for a c...,0.807779,0.738570,0.771626
5,What are some in-demand technical skills for a...,0.602340,0.657617,0.628766
6,What are the career prospects for individuals ...,0.706135,0.722272,0.714112
7,What are new job roles created in the AI indus...,0.622218,0.623666,0.622942
8,What are the main responsibilities of a web de...,0.589469,0.678378,0.630806
9,What is the job of a software quality assuranc...,0.742613,0.804609,0.772369


In [14]:
generate_bert_results_table(fine_data_cs_industry)

,question,precision,recall,f1
0,What are the latest trends in artificial intel...,0.842354,0.777274,0.808507
1,What are some emerging programming languages t...,0.831567,0.786275,0.808287
2,I am a beginner that wants to get into Data Sc...,0.820513,0.791951,0.805979
3,I am a final-year Computer Science student wan...,0.856178,0.776528,0.814410
4,What are the essential skills required for a c...,0.765383,0.789562,0.777284
5,What are some in-demand technical skills for a...,0.887392,0.840082,0.863089
6,What are the career prospects for individuals ...,0.865855,0.814591,0.839441
7,What are new job roles created in the AI indus...,0.806929,0.777754,0.792073
8,What are the main responsibilities of a web de...,0.754864,0.838388,0.794437
9,What is the job of a software quality assuranc...,0.824233,0.864863,0.844060


Finetuning doesn't reduce hallucinations and provide timely-relevant outputs!


### Comparing (no RAG) with final pipeline (with RAG)

In [7]:
import pandas as pd
pipeline_eval = pd.DataFrame({
    "question": [
        "Can you give me a typical job description of graduate role for Software Engineering?",
        "Can you give an example job description for a Software Engineer intern?",
        "What particular skills do recruiters look for in a Web developer?",
        "What are cybersecurity analyst qualifications that recruiters look for?",
        "What are some software tools that an IT consultant need to know?",
        "What are skill descriptions most recruiters look for in a software engineer?",
        "What are some programming languages demanded in web developer jobs?",
        "Can you give common job requirements in Computer Vision?",
        "Please explain the responsibilities of a software architect.",
        "What qualifications does a full-stack engineer need?",
        "What responsibilities would I have as an AI solutions architect?",
        "What are recent topics a Cybersecurity consultant should learn about?",
        "Give me some fairly recent topics in the realm of data science.",
        "Can you explain the skills or experiences that recruiters look for in a Software Architect?",
        "What programming skills would IT managers require to possess?",
    ]
})

pipeline_eval2 = pd.DataFrame({
    "question": [
        "What programming languages should a software engineer know these days?",
        "Please explain to me skills I need to learn to become a cloud engineer.",
        "What are the responsibilities for a typical IT manager?",
        "What are examples of frameworks I would be working on as a Database Administrator?",
        "Can you explain some skills or qualifications to become a game developer?",
        "What are the special skills a professional working in cybersecurity should have?",
        "What does a typical UX designer do?",
        "What do recruiters look for in an AI engineer?",
        "If I want to get a graduate role in game developing, what skills should I expand on?",
        "Based on job descriptions, what does a software architect do?",
        "If I want to become a software anlayst, please advise me where to start.",
        "Explain several frameworks an AI engineer need to know how to use.",
        "Please describe the responsibilities of a UX designer",
        "Advise on what to learn to become a proficient cloud engineer.",
        "What are characteristics that recruiters typically look for in a database administrator"
    ]
})

In [8]:
fine_eval1 = chain_predictions_from_data(fine_tuned_llm_chain, pipeline_eval)

In [10]:
fine_eval2 = chain_predictions_from_data(fine_tuned_llm_chain, pipeline_eval2)

In [9]:
fine_eval1

,question,predictions
0,Can you give me a typical job description of g...,"\nAs a software engineer, you will be responsi..."
1,Can you give an example job description for a ...,\nWe are looking for a motivated and passionat...
2,What particular skills do recruiters look for ...,\nRecruiters typically look for Web developers...
3,What are cybersecurity analyst qualifications ...,\nRecruiters in the cybersecurity industry typ...
4,What are some software tools that an IT consul...,\nAn IT consultant should have knowledge of va...
5,What are skill descriptions most recruiters lo...,\nRecruiters often seek software engineers wit...
6,What are some programming languages demanded i...,"\nAs a web developer, some of the programming ..."
7,Can you give common job requirements in Comput...,\nSure! Here are some common job requirements ...
8,Please explain the responsibilities of a softw...,"\nAs a software architect, it is my responsibi..."
9,What qualifications does a full-stack engineer...,\nA full-stack engineer should have a strong f...


In [11]:
fine_eval2

,question,predictions
0,What programming languages should a software e...,"\nAs a software engineer, it is important to h..."
1,Please explain to me skills I need to learn to...,"\nTo become a cloud engineer, you should learn..."
2,What are the responsibilities for a typical IT...,"\nAs an IT manager, the primary responsibiliti..."
3,What are examples of frameworks I would be wor...,"\nAs a Database Administrator, some examples o..."
4,Can you explain some skills or qualifications ...,"\nTo become a game developer, it is important ..."
5,What are the special skills a professional wor...,\nAs a professional in the field of cybersecur...
6,What does a typical UX designer do?,\nA typical UX designer is responsible for cre...
7,What do recruiters look for in an AI engineer?,\nRecruiting AI engineers typically look for c...
8,If I want to get a graduate role in game devel...,"\nAs a game developer, it's essential to have ..."
9,"Based on job descriptions, what does a softwar...",\nA software architect is a professional who d...


In [12]:
fine_eval1.to_csv("no_rag1.csv", index=False)
fine_eval2.to_csv("no_rag2.csv", index=False)